In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
#print('导出成功')

from bokeh.io import output_notebook
output_notebook()
#导入notrbook绘图模块

from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource
#导入图表绘制、图表展示模块
#导入ColumnDataSource模块

Loading BokehJS ...

In [12]:
#E:\视频qyp\【瑞客论坛 www.ruike1.com】网易云课堂 数据分析师Python进阶（完结）\00-0 QQ群资料\【非常重要】课程资料\CLASSDATA_ch06数据分析项目实战\练习04_电商打折套路解析
import os
os.chdir('E:/视频qyp/【瑞客论坛 www.ruike1.com】网易云课堂 数据分析师Python进阶（完结）/00-0 QQ群资料/【非常重要】课程资料/CLASSDATA_ch06数据分析项目实战/练习04_电商打折套路解析')

df = pd.read_excel('双十一淘宝美妆数据.xlsx',index_col=0)  #将update_time变为时间序列,index_col=0
### 时间序列
df.head(1)


,id,title,price,店名
update_time,,,,
2016-11-14,A2251521828,相宜本草四倍多萃润泽精华保湿霜滋润面霜女补水,79.0,相宜本草


In [6]:
len(df)


27642

In [7]:
df_columns = df.columns.tolist()
df_columns

['update_time', 'id', 'title', 'price', '店名']

In [13]:
#清洗
df.fillna(0,inplace = True)  #将空值替换为0
df['date'] = df.index.day  #提取销售日期
print('数据量为%i条'  % len(df))
print('数据时间周期为：%n', df.index.unique())
df.head(2)


数据量为27642条
数据时间周期为：%n DatetimeIndex(['2016-11-14', '2016-11-13', '2016-11-12', '2016-11-11',
               '2016-11-10', '2016-11-09', '2016-11-08', '2016-11-07',
               '2016-11-06', '2016-11-05'],
              dtype='datetime64[ns]', name='update_time', freq=None)


,id,title,price,店名,date
update_time,,,,,
2016-11-14,A2251521828,相宜本草四倍多萃润泽精华保湿霜滋润面霜女补水,79.0,相宜本草,14
2016-11-14,A2251610120,正品相宜本草四倍多萃润泽洁面膏深层清洁滋润补水保湿,29.0,相宜本草,14


In [18]:
# 双十一当天在售的商品占比情况
# 按照商品销售节奏分类，我们可以将商品分为7类
#   A. 11.11前后及当天都在售 → 一直在售
#   B. 11.11之后停止销售 → 双十一后停止销售
#   C. 11.11开始销售并当天不停止 → 双十一当天上架并持续在售
#   D. 11.11开始销售且当天停止 → 仅双十一当天有售
#   E. 11.5 - 11.10 → 双十一前停止销售
#   F. 仅11.11当天停止销售 → 仅双十一当天停止销售
#   G. 11.12开始销售 → 双十一后上架

data1 = df[['id','title','店名','date']]
data1.head()

,id,title,店名,date
update_time,,,,
2016-11-14,A2251521828,相宜本草四倍多萃润泽精华保湿霜滋润面霜女补水,相宜本草,14
2016-11-14,A2251610120,正品相宜本草四倍多萃润泽洁面膏深层清洁滋润补水保湿,相宜本草,14
2016-11-14,A2251615486,相宜本草红石榴亮白嫩肤水女紧致补水保湿精华爽肤水女秋冬护肤品,相宜本草,14
2016-11-14,A2252191048,相宜本草红石榴鲜活透亮面贴膜7片补水保湿女化妆品滋润,相宜本草,14
2016-11-14,A2253094636,相宜本草去死皮膏洁净面部全身温和去角质补水化妆,相宜本草,14


In [20]:
#统计不同商品的销售开始日期，截止日期

d1 = data1[['id','date']].groupby(by = 'id').agg(['min','max'])['date']
d1.head()


,min,max
id,,
A10027317366,5,14
A10588608182,5,14
A10847151685,5,14
A12229499633,5,10
A12229615671,5,11


In [28]:
#筛选双十一当天售卖的商品
id_11 = data1[data1['date']==11]['id'].unique()
#id_11
d2=pd.DataFrame({'id':id_11,'双十一当天是否售卖':True})
d2.head()

,id,双十一当天是否售卖
0,A2251570112,True
1,A2251615486,True
2,A2252191048,True
3,A2253281880,True
4,A2253282718,True


In [29]:
id_date = pd.merge(d1,d2,left_index=True,right_on='id',how = 'left')
id_date['双十一当天是否售卖'][id_date['双十一当天是否售卖']!=True] = False
#print(id_date.head())
# 合并数据

m = len(data1['id'].unique())
m_11 = len(id_11)
m_11_pre = m_11/m
print('商品总数为%i个\n-------' % m)
print('双十一当天参与活动的商品总数为%i个，占比为%.2f%%\n-------' % (m_11,m_11_pre*100))
print('品牌总数为%i个\n' % len(data1['店名'].unique()),data1['店名'].unique())
# 统计


商品总数为3502个
-------
双十一当天参与活动的商品总数为2335个，占比为66.68%
-------
品牌总数为22个
 ['相宜本草' '佰草集' '欧莱雅' '美宝莲' '玉兰油' '蜜丝佛陀' '悦诗风吟' 'SKII' '兰芝' '妮维雅' '自然堂' '倩碧'
 '欧珀莱' '美加净' '雅诗兰黛' '资生堂' '兰蔻' '雅漾' '雪花秀' '植村秀' '薇姿' '娇兰']


In [1]:
id_date['type'] = '待分类'
id_date['type'][(id_date['min'] <11)&(id_date['max']>11)] = 'A'      #  A类：11.11前后及当天都在售 → 一直在售
id_date['type'][(id_date['min'] <11)&(id_date['max']==11)] = 'B'     #  B类：11.11之后停止销售 → 双十一后停止销售
id_date['type'][(id_date['min'] ==11)&(id_date['max']>11)] = 'C'     #  C类：11.11开始销售并当天不停止 → 双十一当天上架并持续在售
id_date['type'][(id_date['min'] ==11)&(id_date['max']==11)] = 'D'    #  D类：11.11开始销售且当天停止 → 仅双十一当天有售
id_date['type'][id_date['双十一当天是否售卖']== False] = 'F'         #  F类：仅11.11当天停止销售 → 仅双十一当天停止销售
id_date['type'][id_date['max']<11] = 'E'                             #  E类：11.5 - 11.10 → 双十一前停止销售
id_date['type'][id_date['min'] >11] = 'G'                            #  G类：11.11之后开始销售 → 双十一后上架
# 商品销售节奏分类


NameError: name 'id_date' is not defined

In [ ]:
#结束
